### Tensor

In [ ]:
import torch

Podemos verificar si tenemos un dispositivo disponible con cuda `torch.cuda.is_available()`, es posible crear un tensor apartir de funciones randomicas `torch.rand(n,m)` 



In [ ]:
print(torch.cuda.is_available())
print(torch.rand(2,2))

True
tensor([[0.6841, 0.0154],
        [0.5967, 0.3669]])


Crear un tensor apartir de una lista

In [ ]:
x = torch.tensor( [[1,0,0],
                   [0,1,0],
                   [0,0,1]])
print(x)

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])


Podemos cambiar un elemento en un tensor usando la indexación estándar de Python:

In [ ]:
x[2][2] = 5
print(x)

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 5]])


Puede utilizar funciones especiales de creación para generar tipos particulares de tensores. En particular,
`ones()` y `zeroes()` generarán tensores llenos de 1s y 0s, respectivamente:

In [ ]:
z=torch.zeros(2,2)
w=torch.ones(2,2)

print("Tensor de ceros: ", z)
print("Tensor de unos: ", w)

Tensor de ceros:  tensor([[0., 0.],
        [0., 0.]])
Tensor de unos:  tensor([[1., 1.],
        [1., 1.]])


Se puede realizar operaciones estandar de matematicas con tensores

In [ ]:
torch.ones(2,2) +torch.ones(2,2)

tensor([[2., 2.],
        [2., 2.]])

Y si tienes un tensor de rango 0, puedes sacar el valor con `item()`, note la diferencia entre los dos print statement

In [ ]:
x=torch.tensor([2])
print(x)
print(x.item())

tensor([2])
2


Los tensores pueden convivir entre CPU y GPU, con la funcion `to()`

In [ ]:
 cpu_tensor =torch.rand(2)
 cpu_tensor.device

device(type='cpu')

In [ ]:
gpu_tensor = cpu_tensor.to("cuda")
gpu_tensor.device

device(type='cuda', index=0)

Se recomienda hechar un vistazo a la documentacion de [PyTorch](https://pytorch.org/docs/stable/index.html) para más informacion de funciones que podrias aplicar.

### Operaciones Tensor

Una de las operaciones más usadas esa encontrar el valor maximo de un tensor al igual que el indice de este, ya que a menudo corresponde a la clase que el neural que la red ha decidido en su predicción final.
Esto se puede hacer con el `max()` y las funciones de `argmax()`. También podemos usar `item()` para extraer un valor estándar de Python
de un tensor 1D.

In [69]:
x=torch.rand(3,3)
print(x)
x.max()

tensor([[0.5622, 0.5091, 0.6377],
        [0.7073, 0.0573, 0.7528],
        [0.9375, 0.8666, 0.7508]])


tensor(0.9375)

A veces, Necesitamos cambiar el tipo de tensor; por ejemplo, de un tensor largo a un Tensor Flotante. 
Podemos hacer esto con `to()`:

In [75]:
long_tensor = torch.tensor([[1,0,0],[0,1,0],[0,0,1]])
print(long_tensor.type())
long_tensor

torch.LongTensor


tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

In [82]:
float_tensor = long_tensor.to(dtype=torch.float32)
print(float_tensor.type())
float_tensor

torch.FloatTensor


tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

La mayoría de las funciones que operan en un tensor y devuelven un tensor crean un nuevo tensor para almacenar el resultado. 
Sin embargo, si quiere guardar la memoria, mire a ver si se define una función *in-place*, que debería tener el mismo nombre que la función original pero con un underscore (_).

In [85]:
random_tensor = torch.rand(2,2)
print(random_tensor)
random_tensor.log2()

tensor([[0.8169, 0.0371],
        [0.3874, 0.9689]])


tensor([[-0.2918, -4.7528],
        [-1.3681, -0.0455]])

In [86]:
random_tensor.log2_() # Aqui se aplica

tensor([[-0.2918, -4.7528],
        [-1.3681, -0.0455]])

Otra operación común es (*reshaping*) la remodelación de un tensor.Esto puede ocurrir a menudo porque la capa de una red neural puede requerir una forma de entrada un tanto diferente a la que se posee.

Por ejemplo, el dataset MNIST, fue escrito a mano en una serie de imagenes tamaño de 28 × 28, pero esto fue empaquetado en arrays de longitud 784. 
Para usar las redes que vamos a construir, necesitamos convertirlas de nuevo en tensores de 1 × 28 × 28 (el 1, es el número de canales - normalmente es rojo, verde y azul pero como los dígitos del MNIST estan en escala de grises, sólo tenemos un canal). Podemos hacer esto con `view()` o `reshape()`:

In [104]:
flat_tensor = torch.rand(784) #creamos un tensor con valores aleatorias de longitud 784
print("Tensor rand: ",flat_tensor.shape)
viewed_tensor = flat_tensor.view(1,28,28)
print("Tensor with view: ", viewed_tensor.shape)

Tensor rand:  torch.Size([784])
Tensor with view:  torch.Size([1, 28, 28])


In [103]:
reshaped_tensor = flat_tensor.reshape(1,28,28)
print("Tensor rand: ",flat_tensor.shape)
print("Tensor with reshape: ", reshaped_tensor.shape)

Tensor rand:  torch.Size([784])
Tensor with reshape:  torch.Size([1, 28, 28])


Note que la forma del tensor reformado tiene que tener el mismo número de elementos totales como el original. Si intentas con `flat_tensor.reshape(3,28,28)`, vera  un error como este:

In [102]:
flat_tensor.reshape(3,28,28)

RuntimeError: ignored

Ahora se preguntarán cuál es la diferencia entre `view() `y `reshape()`. La respuesta es que` view()` opera como una vista en el tensor original, de modo que si los datos subyacentes se cambian, la vista también cambiará (y viceversa). Sin embargo, `view()` puede
arrojar errores si la vista requerida no es contigua; es decir, no comparte el mismo bloque de memoria que ocuparía si se creara un nuevo tensor de la forma requerida desde el principio. Si esto ocurre, tienes que llamar: `tensor.contiguous()` antes de que uses `view()`.
Así que en general, recomiendo usar` reshape()`.

Finalmente, puede que necesite re-organizar las dimensiones de un tensor. Esto es más comun en imagenes que son almacenadas como: [ height, width, channel ] tensores, esto esta bien, pero PyTorch prefiere tratarlos: [channel, height, width].
Para esto es posible usar `permute()`, donde los ordena segun la indexación, es ideal para tratarlo de forma directa.

In [109]:
hwc_tensor = torch.rand(640,480,3)
print(hwc_tensor.shape)
chw_tensor = hwc_tensor.permute(2,0,1)
chw_tensor.shape

torch.Size([640, 480, 3])


torch.Size([3, 640, 480])

En este momento tenemos todas las herramientas necesarias para construir redes neuronales en PyTorch